In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [2]:
data_dir = '/gdrive/My Drive/colab/week4/data'

In [113]:
import re
import operator
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [4]:
import os
from tqdm import tqdm #progress bar를 보여주기 위함

# line별로 읽어서 문장과 라벨을 나눔
def load_data(file_path):
  file = open(os.path.join(data_dir, file_path))

  texts, labels = [], []

  for line in tqdm(file.readlines()):
    text, label = line.strip().split('\t')
    texts.append(text)
    labels.append(label)
  return texts, labels

In [549]:
train_X, train_y = load_data('sentiment_train.txt')
test_X, test_y = load_data('sentiment_test.txt')

100%|██████████| 300/300 [00:00<00:00, 168445.94it/s]


In [19]:
print(train_X[0])

제품 도 너무 너무 맘 에 들 ㄴ답니다


In [6]:
print(len(train_X))
print(len(train_y))

2686
2686


# Train Data 전처리하기

In [533]:
#완성형 한글만 데이터로 사용
def cleanText(document):
  for i in range(len(document)):
    document[i] = re.sub('[^가-힣\s]','',document[i])
  return document

In [550]:
clean_train_X = cleanText(train_X)

In [49]:
print(train_X[0])

제품 도 너무 너무 맘 에 들 답니다


# label을 이용해서 긍정문서와 부정문서를 나눈다. 

In [50]:
from operator import eq
import re
def split_train(train_X,train_y):
  i=0
  p_train, n_train = [],[]
  for i in range(len(train_X)):
    if 'P' in train_y[i]:
      p_train.append(train_X[i])
    elif 'N' in train_y[i]:
      n_train.append(train_X[i])
  return p_train, n_train

In [551]:
p_train, n_train = split_train(clean_train_X,train_y)

긍정문서와 부정문서로 부터 각각의 단어장을 구하고 tf vector를 구한다. 

In [577]:
count_vectorizer1 = CountVectorizer()
count_vectorizer2 = CountVectorizer()
p_count_vectorizer = count_vectorizer1.fit_transform(p_train).toarray()
n_count_vectorizer = count_vectorizer2.fit_transform(n_train).toarray()

In [578]:
p_vocabulary = list(sorted(count_vectorizer1.vocabulary_.keys()))
n_vocabulary = list(sorted(count_vectorizer2.vocabulary_.keys()))

In [581]:
print(len(p_vocabualry))
print(p_count_vectorizer.shape)

1084
(1084,)


In [580]:
p_tf = dict.fromkeys(p_vocabulary,0)
n_tf = dict.fromkeys(n_vocabulary,0)
p_count_vectorizer = np.sum(p_count_vectorizer,axis =0)
n_count_vectorizer = np.sum(n_count_vectorizer,axis=0)

In [582]:
for i,key in enumerate(p_tf):
  p_tf[key] = p_count_vectorizer[i]

for i,key in enumerate(n_tf):
  n_tf[key] = n_count_vectorizer[i]

In [584]:
print(p_tf['제품'])

65


# TF-IDF 구하기 
tf는 CountVectorizer이용

idf는 따로 함수를 구현

idf를 구할 때의 공식은 log10(2/df값)을 이용

In [587]:
import math

def get_df(ptfv,ntfv,vocab):
  dfv = dict.fromkeys(vocab,0)
  for d in dfv:
    if d in ptfv:
      dfv[d]+=1
    if d in ntfv:
      dfv[d]+=1
  return dfv

def get_idf(ptfv,ntfv,vocab):
  idfv = get_df(ptfv,ntfv,vocab)
  for key, val in idfv.items():
    idfv[key] = math.log10(2/float(val))
  
  return idfv

In [588]:
p_idfv = get_idf(p_tf,n_tf,p_vocabulary)
n_idfv = get_idf(p_tf,n_tf,n_vocabulary)

In [590]:
#tf와 idfv곱하기
p_tfidf = dict.fromkeys(p_vocabualry,0)
n_tfidf = dict.fromkeys(n_vocabulary,0)
for i,key in enumerate(p_vocabulary):
  p_tfidf[key] = p_tf[key]*p_idfv[key]

for i,key in enumerate(n_vocabulary):
  n_tfidf[key] = n_tf[key]*n_idfv[key]

# 긍정과 부정에서 각각 상위 100개의 단어 구하기

In [591]:
p_dict = sorted(dict.items(p_tfidf),key=operator.itemgetter(1),reverse=True)
p_dict = dict((x,y) for x,y in p_dict)

n_dict = sorted(dict.items(n_tfidf),key=operator.itemgetter(1),reverse=True)
n_dict= dict((x,y) for x,y in n_dict)

0.0


In [593]:
#긍정과 부정 단어 각각 100개 추출
p_list = list(p_dict.keys())
n_list = list(n_dict.keys())

p_list = p_list[0:100]
n_list = n_list[0:100]

np_vocab =list(set(p_list).union(set(n_list)))

In [594]:
print(len(np_vocab))
print(np_vocab)

200
['욕실', '강원도', '수시', '무겁', '한가지', '장난', '독신자', '사고', '불안', '가격좋고디자인수려하고대용량이라정말편리하고좋습니다', '저녁', '교환', '다는데', '상하', '짜증', '그치', '어른', '강츄', '기쁘', '지르', '감사드림니다', '호스', '한테', '넉넉', '성의', '가게', '강추예요', '결제', '가격싸고', '어쩌', '쇼핑몰', '관리법', '감솨', '결국', '따로', '사시', '길이', '높이', '관련', '자체', '언제', '섬유유연제가', '화가', '구조', '라는', '강추입니다', '기본', '오히려', '자꾸', '제일', '갓난아이', '꼼꼼', '무조건', '기원', '장마철', '그런데', '개인', '질문', '똑같', '사도', '보내', '화이트', '반품', '완벽', '강추에요', '게시판', '가격땜에', '문의', '원래', '진동', '외부', '가동', '나가', '구김', '답변', '가구', '토요일', '세심', '감안', '감사하드라구요', '대충', '신속', '국산', '아쉬움', '어쨌든', '고민', '대단히', '주정', '베리', '다행', '기숙사', '아닌가요', '얘기', '단점', '대우전자', '나요', '정신', '나이드신', '값싸', '중요', '엉킴도', '업체', '공기방울', '이서', '어떻', '훌륭', '가격만족성능만족배송만족서어비스만족모두가좋네요', '무슨', '화요일', '배려', '적극', '재고', '노인', '가르치', '으니까', '여러', '좋구여', '몇일', '뒤지', '주신', '맞히', '쿠폰', '떨어지', '시장', '냐고', '가격까지맘에', '어렵', '하하', '사용법', '사과', '는다는', '저렵', '약하', '한마디', '가격배송폐가전수거', '전용', '새해', '굳이', '불만', '전기', '당일', '관계', '폐가', '강추합니다', '안심', '이해',

In [ ]:
np_vocab이 없는 문장은 제거해서 train한다. 

In [595]:
#행의 합이 0인 array는 제거
new_train_X=[]
new_train_y=[]

In [596]:
for i,sentence in enumerate(train_X):
  new_sent=''
  for j, word in enumerate(np_vocab):
    if word in sentence:
     new_sent += word + ' '
  if new_sent.strip()!='':
    new_train_X.append(new_sent)
    new_train_y.append(train_y[i])

In [597]:
print(len(new_train_X),len(new_train_y))
vectorizer = CountVectorizer()
feature_train_X = vectorizer.fit_transform(new_train_X)

741 741


# 학습!!

In [598]:
classifier = MultinomialNB()
classifier.fit(feature_train_X,new_train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Test_X의 tf-idf구하기

In [599]:
new_test_X=[]
new_test_y=[]
for i,sentence in enumerate(test_X):
  new_sent=''
  for j, word in enumerate(np_vocab):
    if word in sentence:
     new_sent += word + ' '
  if new_sent.strip()!='':
    new_test_X.append(new_sent)
    new_test_y.append(test_y[i])

In [600]:
print(len(new_test_X),len(new_test_y))

90 90


In [601]:
feature_test_X = vectorizer.transform(new_test_X)

In [602]:
predictions = classifier.predict(feature_test_X).tolist()

In [603]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.2f' % accuracy_score(new_test_y, predictions))

Accuracy: 0.87


In [604]:
for i in range(5):
  print(test_y[i],predictions[i])

<P> <N>
<N> <P>
<P> <N>
<N> <N>
<P> <N>
